In [1]:
from CNN import CNN
from Utils import *
import os
from torch.utils.data import DataLoader

PATH_PROJECT_ROBIN = "/home/robin/Bureau/ETUDES/M2/S2/TAP/"
PATH_PROJECT_QUENTIN = "..."
PATH_PROJECT_PERSONNE = PATH_PROJECT_ROBIN
PATH_RAW_DATASET = "Projet-Traitement-Automatique-de-la-Parole/Dataset/"
GLOBAL_PATH_DATASET = os.path.join(PATH_PROJECT_PERSONNE, PATH_RAW_DATASET)
GLOBAL_PATH_RAW_DATASET = os.path.join(GLOBAL_PATH_DATASET, "speech_commands_v0.02/")
GLOBAL_PATH_BASIC_DATASET = os.path.join(GLOBAL_PATH_DATASET, "Dataset_basique/")
filename = os.path.join(GLOBAL_PATH_DATASET, "dataset_basique.pkl")

In [2]:
with open(filename, "rb") as f:
    data = pickle.load(f)

In [3]:
x_train, y_train, metadata_train = data["train"]["signals"][:200], data["train"]["labels"][:200], data["train"]["metadata"][:200]
x_val, y_val, metadata_val = data["val"]["signals"][:200], data["val"]["labels"][:200], data["val"]["metadata"][:200]
x_test, y_test, metadata_test = data["test"]["signals"][:200], data["test"]["labels"][:200], data["test"]["metadata"][:200]

train_dataset = SpeechCommandDataset(x_train, y_train, metadata_train, transform_type="MFCC")
val_dataset = SpeechCommandDataset(x_val, y_val, metadata_val, transform_type="MFCC")
test_dataset = SpeechCommandDataset(x_test, y_test, metadata_test, transform_type="MFCC")

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)

torch.Size([200, 1, 40, 97])
torch.Size([200, 1, 40, 97])
torch.Size([200, 1, 40, 97])


In [12]:
model = CNN(1,6)
_ = train(model,train_loader,val_loader, nb_steps=1000)